In [1]:
import tensorflow as tf


def reduce_logsumexp(input_tensor, reduction_indices=1, keep_dims=False):
    """Computes the sum of elements across dimensions of a tensor in log domain.

     It uses a similar API to tf.reduce_sum.

    Args:
    input_tensor: The tensor to reduce. Should have numeric type.
    reduction_indices: The dimensions to reduce. 
    keep_dims: If true, retains reduced dimensions with length 1.
    Returns:
    The reduced tensor.
    """
    
    max_input_tensor1 = tf.reduce_max(
        input_tensor, reduction_indices, keep_dims=keep_dims)
    max_input_tensor2 = max_input_tensor1
    
    if not keep_dims:
        max_input_tensor2 = tf.expand_dims(max_input_tensor2, reduction_indices)
        
    return tf.log(
        tf.reduce_sum(
            tf.exp(input_tensor - max_input_tensor2),
            reduction_indices,
            keep_dims=keep_dims)) + max_input_tensor1


def logsoftmax(input_tensor):
    """Computes normal softmax nonlinearity in log domain.

     It can be used to normalize log probability.
     The softmax is always computed along the second dimension of the input Tensor.     

    Args:
    input_tensor: Unnormalized log probability.
    Returns:
    normalized log probability.
    """
    return input_tensor - reduce_logsumexp(input_tensor, reduction_indices=0, keep_dims=True)

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import helper as hlp

# Loading data
#data_in = np.load('data2D.npy')
data_in = np.load('data100D.npy')
[num_pts, dim] = np.shape(data_in)

N = num_pts
D = dim

# Distance function for K-means
def distanceFunc(X, MU):
    # Inputs
    # X: is an NxD matrix (N observations and D dimensions)
    # MU: is an KxD matrix (K means and D dimensions)
    # Outputs
    # pair_dist: is the squared pairwise distance matrix (NxK)
    # TODO

    return tf.reduce_sum(tf.squared_difference(tf.expand_dims(X,1),tf.expand_dims(MU,0)), 2)

def K_Means(k, is_valid=False):
    # For Validation set
    if is_valid:
        valid_batch = int(num_pts / 3.0)
        train_batch = N - valid_batch
        np.random.seed(45689)
        rnd_idx = np.arange(num_pts)
        np.random.shuffle(rnd_idx)
        val_data = data_in[rnd_idx[:valid_batch]]
        data = data_in[rnd_idx[valid_batch:]]
    else:
        data = data_in

    MU = tf.Variable(tf.random_normal([k, D]))
    X = tf.placeholder(tf.float32, shape = [None, D])

    dist_matrix = distanceFunc(X, MU)
    dist = tf.reduce_min(dist_matrix, axis = 1)
    cluster = tf.argmin(dist_matrix, axis = 1)
    _, _, count = tf.unique_with_counts(cluster)
    percentage = tf.divide(count,N)

    loss_mu = tf.reduce_sum(dist)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.99, epsilon=1e-5)
    optimizer = optimizer.minimize(loss_mu)

    train_record = []

    init = tf.global_variables_initializer()
    session = tf.InteractiveSession()
    session.run(init)

    for i in range(200):
        _, loss = session.run([optimizer, loss_mu], feed_dict = {X: data})
        train_record.append(loss)

    clusters, percentages = session.run([cluster, percentage], feed_dict = {X: data})

    if is_valid is True:
        valid_loss = session.run([loss_mu], feed_dict = {X: val_data})

        session.close()
        return valid_loss

    else:
        plt.figure()
        plt.title('Loss with number of clusters = {}'.format(k))
        plt.plot(train_record,'')
        plt.xlabel('epochs')
        plt.ylabel('loss')
        plt.grid()
        plt.legend(['Training Loss'])
        plt.savefig('Training_Loss_k={}.png'.format(k))

        print(percentages)

        plt.figure()
        plt.title('Data Points with number of clusters = {}'.format(k))
        plt.scatter(data_in[:,0], data_in[:,1], c=clusters, s=0.5, alpha=0.8)
        plt.xlabel('Dimension 1')
        plt.ylabel('Dimension 2')
        plt.grid()
        plt.savefig('Data_Points_k={}.png'.format(k))


        session.close()
        return

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import helper as hlp

# Loading data
data = np.load('data100D.npy')
#data = np.load('data2D.npy')
[num_pts, dim] = np.shape(data)

N = num_pts
D = dim

def MoG(k, is_valid=False):
    # For Validation set
    train_batch = N
    train_data = data
    if is_valid:
        valid_batch = int(num_pts / 3.0)
        train_batch = N - valid_batch
        np.random.seed(45689)
        rnd_idx = np.arange(num_pts)
        np.random.shuffle(rnd_idx)
        val_data = data[rnd_idx[:valid_batch]]
        train_data = data[rnd_idx[valid_batch:]]

    Mu = tf.Variable(tf.random.truncated_normal([k, D]))
    X = tf.placeholder(tf.float32, shape = [None, D])
    phi = tf.Variable(tf.random.truncated_normal([k, 1]))
    psi = tf.Variable(tf.random.truncated_normal([k, 1]))
    sigma = tf.exp(phi)
    log_pi = logsoftmax(psi)

    log_Gauss = log_GaussPDF(X,Mu,sigma)
    log_pstr = log_posterior(log_Gauss, log_pi)

    mle_predict = tf.argmax(log_pstr,axis=1)
    logloss = - tf.reduce_sum(reduce_logsumexp(log_Gauss + tf.transpose(log_pi)),axis=0)
    _, _, count = tf.unique_with_counts(mle_predict)
    percentage = tf.divide(count, train_batch)


    optimizer = tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.99, epsilon=1e-5)
    optimizer = optimizer.minimize(logloss)

    train_record = []

    init = tf.global_variables_initializer()
    session = tf.InteractiveSession()
    session.run(init)

    for i in range(200):
        _, loss = session.run([optimizer, logloss], feed_dict = {X: train_data})
        train_record.append(loss)

    predict, percentages = session.run([mle_predict, percentage], feed_dict = {X: train_data})

    if is_valid is True:
        valid_loss = session.run(logloss, feed_dict = {X: val_data})
        session.close()
        return valid_loss

    else:
    
        plt.figure()
        plt.title('Loss with number of gaussian clusters = {}'.format(k))
        plt.plot(train_record,'')
        plt.xlabel('epochs')
        plt.ylabel('loss')
        plt.grid()
        plt.legend(['Training Loss'])
        plt.savefig('Gaussian_Training_Loss_k={}.png'.format(k))

        plt.figure()
        plt.title('Data Points with number of gaussian clusters = {}'.format(k))
        plt.scatter(data[:,0], data[:,1], c=predict, s=0.5, alpha=0.8)
        plt.xlabel('Dimension 1')
        plt.ylabel('Dimension 2')
        plt.grid()
        plt.savefig('Gaussian_Data_Points_k={}.png'.format(k))
        

        print("Final value of phi and psi:")
        print(phi.eval())
        print(psi.eval())

        print("Final value of sigma and pi:")
        print(sigma.eval())
        print(tf.exp(log_pi).eval())


        session.close()
        return


# Distance function for GMM
def distanceFunc(X, MU):
    # Inputs
    # X: is an NxD matrix (N observations and D dimensions)
    # MU: is an KxD matrix (K means and D dimensions)
    # Outputs
    # pair_dist: is the pairwise distance matrix (NxK)
    # TODO
    return tf.reduce_sum(tf.squared_difference(tf.expand_dims(X,1),tf.expand_dims(MU,0)), 2)

def log_GaussPDF(X, mu, sigma):
    # Inputs
    # X: N X D
    # mu: K X D
    # sigma: K X 1

    # Outputs:
    # log Gaussian PDF N X K

    distance = distanceFunc(X,mu)
    exp = - tf.divide(distance, 2 * tf.transpose(sigma))
    coef = - (D / 2) * tf.log(2 * np.pi) - (1/2) * tf.log(tf.transpose(sigma))
    return tf.add(coef, exp)



def log_posterior(log_PDF, log_pi):
    # Input
    # log_PDF: log Gaussian PDF N X K
    # log_pi: K X 1

    # Outputs
    # log_post: N X K

    # TODO
    num = tf.add(log_PDF, tf.transpose(log_pi))
    den = reduce_logsumexp(num, keep_dims=True)
    return tf.subtract(num,den)

In [4]:
print('K-Means:')
print('k = 5, valid loss = ', K_Means(5, True))
print('k = 10, valid loss = ', K_Means(10, True))
print('k = 15, valid loss = ', K_Means(15, True))
print('k = 20, valid loss = ', K_Means(20, True))
print('k = 30, valid loss = ', K_Means(30, True))


print('MoG:\nk = 5, valid loss = ', MoG(5, True))
print('k = 10, valid loss = ', MoG(10, True))
print('k = 15, valid loss = ', MoG(15, True))
print('k = 20, valid loss = ', MoG(20, True))
print('k = 30, valid loss = ', MoG(30, True))

K-Means:
k = 5, valid loss =  [71794.9]
k = 10, valid loss =  [152839.23]
k = 15, valid loss =  [70407.336]
k = 20, valid loss =  [69752.516]
k = 30, valid loss =  [68940.375]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
MoG:
k = 5, valid loss =  315629.84
k = 10, valid loss =  315623.9
k = 15, valid loss =  315639.34
k = 20, valid loss =  315623.5
k = 30, valid loss =  315624.03
